# Telecom Churn Case Study


# Problem Statement

## Business problem overview

. In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.

. For many incumbent operators, retaining high profitable customers is the number one business goal.

. To reduce customer churn, telecom companies need to predict which customers are at high risk of churn.

. In this project, we will analyse customer-level data of a leading telecom firm, build predictive models to identify customers at high risk of churn and identify the main indicators of churn.

# Definitions of churn
. There are various ways to define churn, such as:

## Revenue-based churn:
. Customers who have not utilised any revenue-generating facilities such as mobile internet, outgoing calls, SMS etc. over a given period of time. One could also use aggregate metrics such as ‘customers who have generated less than INR 4 per month in total/average/median revenue’.

The main shortcoming of this definition is that there are customers who only receive calls/SMSes from their wage-earning counterparts, i.e. they don’t generate revenue but use the services. For example, many users in rural areas only receive calls from their wage-earning siblings in urban areas.

## Usage-based churn:
Customers who have not done any usage, either incoming or outgoing - in terms of calls, internet etc. over a period of time.

A potential shortcoming of this definition is that when the customer has stopped using the services for a while, it may be too late to take any corrective actions to retain them. For e.g., if you define churn based on a ‘two-months zero usage’ period, predicting churn could be useless since by that time the customer would have already switched to another operator.

In this project, we will use the usage-based definition to define churn.

## Objective
. To Predict the customers who are about to churn from a telecom operator . Business Objective is to predict the High Value Customers only . We need to predict Churn on the basis of Action Period (Churn period data needs to be deleted after labelling) Churn would be based on Usage

## Requirement:
. Churn Prediction Model . Best Predictor Variables

## Steps to Approach The Best Solution For This Case Study
There are mainly 6 steps

#### Step 1 :
. Data reading . Data Understanding . Data Cleaning Imputing missing values

#### Step-2 :
Need to Filter high value customers

#### Step-3 :
Derive churn need to Derive the Target Variable

#### Step-4 :
Data Preparation .Derived variable .EDA  

#### Step-5 :
Data modeling
.Split data in to train and test sets .Performing Scaling . Handle class imbalance . Dimensionality Reduction using PCA .Classification models to predict Churn (Use various Models )

#### Step-6 :
.Model Evaluation .Prepare Model for Predictor variables selection (Prepare multiple models & choose the best one)

Finally we need to give best Summarize to the company


# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# To suppress the warnings which will be raised
import warnings
warnings.filterwarnings('ignore')

# Displaying all Columns without restrictions
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
# import required libraries
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from sklearn.decomposition import PCA

from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import FeatureUnion

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report

from sklearn.metrics import roc_auc_score

from imblearn.metrics import sensitivity_specificity_support

from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import cross_val_score

from sklearn.metrics import confusion_matrix

from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.svm import SVC

from sklearn.linear_model import LogisticRegression

from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.feature_selection import RFE

import statsmodels.api as sm

from sklearn.metrics import precision_recall_curve

from sklearn import metrics

from imblearn.over_sampling import SMOTE

from sklearn.decomposition import IncrementalPCA

# Step 1 :

## i.Data reading

## ii.Data Understanding

In [ ]:
# read data
df = pd.read_csv("telecom_churn_data (1).csv")
df.head()

In [ ]:
#Looking at data statistics
df.describe()

In [ ]:
df.shape

In [ ]:
# feature type summary
df.info(verbose=1)

In [ ]:
# Checking for null values
df.isnull().sum()

In [ ]:
# Checking the null value percentage
df.isna().sum()/df.isna().count()*100

In [ ]:
# Checking for the duplicates
df.drop_duplicates(subset=None, inplace=True)
df.shape

In [ ]:
#check the size of data
df.size

In [ ]:
#check the axes of data
df.axes

In [ ]:
#check the dimensions of data
df.ndim

In [ ]:
#list of columns
pd.DataFrame(df.columns)

## iii. Data Cleaning , Imputing missing values¶

Checking the column values where users didn't use the services

In [ ]:
#Segregating the columns related to recharge
rech=[]
for i in df.columns:
    if 'rech' in i:
        rech.append(i)
rech

In [ ]:
#Checking the null values in these columns for each month specifically

for j in [6,7,8,9]:
    print(j,'th month ****************************************')
    for k in rech:
        if str(j) in k:
            print(k,'-> ',df[k].isna().sum())


As the number of null values present in these columns are equal , hence the user hasn't use the service at all.
Thus we can impute it with Zero

In [ ]:
# Taking the columns for imputing with 0
zero_impute=[]
for i in ['date_of_last_rech_data_','total_rech_data_','max_rech_data_','count_rech_2g_','count_rech_3g_','av_rech_amt_data_']:
        for j in [6,7,8,9]:
            zero_impute.append(i+str(j))
zero_impute

In [ ]:
#Imputing the columns with zero
for i in zero_impute:
    df[i].fillna(0,inplace=True)

In [ ]:
#Checking after imputation
df[zero_impute].isna().sum()

In [ ]:
#Checking the percentage of null values in each column
100*(df.isna().sum()/df.shape[0])

In [ ]:
#Segregating the columns which has more than 70 % null values
col_to_drop=[]
for i in df.columns:
    if 100*(df[i].isna().sum()/df.shape[0])>70:
        col_to_drop.append(i)
col_to_drop

In [ ]:
#Dropping the high missing value columns
df.drop(col_to_drop,axis=1,inplace=True)

In [ ]:
#Segregating the date columns
date_col=[]
for i in df.columns:
    if 'date' in i:
        date_col.append(i)
date_col    

In [ ]:
#Dropping the date columns as we have already utilised the information present in date columns for imputation in other columns
df.drop(date_col,axis=1,inplace=True)

In [ ]:
#Checking the shape 
df.shape

In [ ]:
#Checking the null value percentage again
null_per=100*(df.isna().sum()/df.shape[0])
null_per[null_per>0]

In [ ]:
#Segregating the cols
for i in null_per[null_per>0].index.to_list():
    df[i].fillna(df[i].median(),inplace=True)


In [ ]:
#Checking the null value percentages to verify
100*(df.isna().sum()/df.shape[0])

In [ ]:
#Deleting the id columns as it's not required
df.drop(['mobile_number', 'circle_id'],axis=1,inplace=True)

In [ ]:
#Checking the unique values in each column
df.nunique().sort_values()

In [ ]:
#Segregating the columns which has 1 unique value
value_1=[]
for i in df.columns:
    if df[i].nunique()==1:
        value_1.append(i)
value_1

In [ ]:
#Dropping the columns which has 1 unique values
df.drop(value_1,1,inplace=True)

In [ ]:
#Checking the unique values in each column again
df.nunique().sort_values()

In [ ]:
#Checking the shape
df.shape

# Step 2:

## Filter high-value customers

In [ ]:
# Calculating the total data recharge amount in months 6 and 7
df['total_rech_amt_data_6']=df['av_rech_amt_data_6'] * df['total_rech_data_6']
df['total_rech_amt_data_7']=df['av_rech_amt_data_7'] * df['total_rech_data_7']
df['total_rech_amt_data_8']=df['av_rech_amt_data_8'] * df['total_rech_data_8']

# Calculating the total recharge amount in months 6 and 7
df['overall_rech_amt_6'] = df['total_rech_amt_data_6'] + df['total_rech_amt_6']
df['overall_rech_amt_7'] = df['total_rech_amt_data_7'] + df['total_rech_amt_7']
df['overall_rech_amt_8'] = df['total_rech_amt_data_8'] + df['total_rech_amt_8']

As we derived the new features for the good phase, we drop the original features to avoid redundancy

In [ ]:
# dropping the corresponding original features
df.drop(['total_rech_amt_data_6','av_rech_amt_data_6', 'total_rech_data_6','total_rech_amt_6',
                  'total_rech_amt_data_7','av_rech_amt_data_7', 'total_rech_data_7','total_rech_amt_7',
                  'total_rech_amt_data_8','av_rech_amt_data_8', 'total_rech_data_8','total_rech_amt_8'],
                  axis=1, inplace=True)

In [ ]:
# Calculating the average recharge done by customer in months June and July(i.e. 6th and 7th month)
df['avg_rech_amt_6_7'] = (df['overall_rech_amt_6'] + df['overall_rech_amt_7'])/2

# Finding the value of 70th percentage to find the HVC
HVC_criteria = df['avg_rech_amt_6_7'].quantile(0.7)
print("\nThe 70th quantile value to determine the High Value Customer is: ",HVC_criteria,"\n")

# Filtering the data to the top 30% considered as High Value Customer
HVC_data = df[df['avg_rech_amt_6_7'] >= HVC_criteria]


The 70th quantile value to determine the High Value Customer is:  478.0 


In [ ]:
# Checking the shape
HVC_data.shape

In [ ]:
## Some of the columns were spelt wrongly for 'vbc_3g_*', lets correct them.
HVC_data.rename(columns = {'aug_vbc_3g':'vbc_3g_8', 'jun_vbc_3g': 'vbc_3g_6', 'jul_vbc_3g' : 'vbc_3g_7', 'sep_vbc_3g' : 'vbc_3g_9'}, inplace = True)

# Step 3:

# Derive churn
. Deriving Churn feature based on Usage based Churn
. Derive churn means hear we are using 9 month(The ‘churn’ phase) data , To get the target variable(In this case stydy they did not provide any target variable we have to derive it from churn phase data) For that, we need to find the derive churn variable using total_ic_mou_9,total_og_mou_9,vol_2g_mb_9 and vol_3g_mb_9 attributes

In [ ]:
# Making a list of all the usage columns
churn_col=['total_ic_mou_9','total_og_mou_9','vol_2g_mb_9','vol_3g_mb_9']

# Creating the Churn column by adding all the churn variables
HVC_data["Churn"] = np.where(HVC_data[churn_col].sum(axis=1) == 0, 1, 0)

In [ ]:
# Checking the Churn feature
HVC_data["Churn"].value_counts(normalize=True) * 100

Removing all the variables related to churn phase

In [ ]:
# Creating list of all columns related to month_9
nine_month_cols=[]

for i in HVC_data.columns:
    if "_9" in i:
        nine_month_cols.append(i)
        
nine_month_cols

In [ ]:
# dropping Month 9 columns

HVC_data=HVC_data.drop(nine_month_cols, axis=1)
HVC_data.shape

In [ ]:
# re-Checking the columns info
HVC_data.info(verbose=True)

In [ ]:
# re-Checking the columns info
HVC_data.info(verbose=True)

In [ ]:
# re-Checking missing Data
missing_data_HVC = pd.DataFrame({"total_missing":HVC_data.isnull().sum() , "perc_missing":HVC_data.isnull().sum()/HVC_data.shape[0]*100})
missing_data_HVC[missing_data_HVC["perc_missing"]>0].sort_values(by="perc_missing", ascending=False)

In [ ]:
# Finding the unique values for all the columns in high value customers data set
#uniqueValues = pd.DataFrame(columns=["Value", "UniqueValueCount"])
for col in HVC_data.columns:
    values = HVC_data[col].value_counts()
    
    print(values)
    print("--------------------------------------------\n")

In [ ]:
HVC_data.shape

In [ ]:
sorted(list(HVC_data.columns))

## finding churn and non churn percentage

In [ ]:
# lets find out churn/non churn percentage
print((HVC_data['Churn'].value_counts()/len(HVC_data))*100)
((HVC_data['Churn'].value_counts()/len(HVC_data))*100).plot(kind="pie")
plt.show()

It is clearly evident from the above pie chart that there is a class imbalance with a churn of nearly 8% and Not_churn of nearly 92%

In [ ]:
HVC_data.shape

## Step 4:
##  Data preparation
## i.Deriving new variables to understand the data
## ii.EDA

The variable 'aon' can be converted as 'tenure_in_months' and store it in seperate dataframe for evaluation

In [ ]:
# creating a new variable 'tenure_in_months'
HVC_data_tenure_in_months = (HVC_data['aon']/30).round(0)

# Since we derived a new column from 'aon', we can drop it
#HVC_data.drop('aon',axis=1, inplace=True)

In [ ]:
# Checking the distribution of 'tenure_in_months'
sns.distplot(HVC_data_tenure_in_months,bins=30)
plt.show()


In [ ]:
tenure_interval = [0, 6, 12, 24, 60, 61]
tenure_label = [ '0-6 Months', '6-12 Months', '1-2 Yrs', '2-5 Yrs', '5 Yrs and above']
HVC_data_tenure_in_months['tenure_range'] = pd.cut(HVC_data_tenure_in_months, tenure_interval, labels=tenure_label)
HVC_data_tenure_in_months['tenure_range'].head()

In [ ]:
# Plotting a barchart for tenure ranges
plt.figure(figsize=[12,8])
sns.barplot(x='tenure_range',y=HVC_data.Churn, data=HVC_data_tenure_in_months)
plt.show()

From the above barchart the following are inferences:

Maximum churns observed for the 0-6 months of tenure
The churning rate decreases gradually as the tenure increases.
Note: On account of gradual (almost linear decrement) of churning rate with the increase of tenure, we prefer to keep the original variable as bucketing does not improve prediction of churn (and hence, we do not merge this dataframe with the original datafame).

Let's aggregate all existing the months of 6 and 7 to a single corresponding columns

In [ ]:
# Checking the columns
sorted(list(HVC_data.columns))

In [ ]:
# creating a list of column names for the month 6 and 7

month_6_cols = [col for col in HVC_data.columns if '_6' in col]
month_7_cols = [col for col in HVC_data.columns if '_7' in col]

In [ ]:
# checking whether the variable exists for both the months 6 and 7:
avg_6_7_columns = []

for col_1 in month_6_cols:
    for col_2 in month_7_cols:
        if col_1[:-2] == col_2[:-2]:
            avg_6_7_columns.append(col_1[:-2] + "_6_7")
set_avg_6_7_columns = set(avg_6_7_columns)
avg_6_7_columns =list(set_avg_6_7_columns)

In [ ]:
for element in avg_6_7_columns:
    if element[-6:] == '_6_6_7':
        avg_6_7_columns.remove(element)

In [ ]:
len(avg_6_7_columns)

In [ ]:
#avg_6_7_columns
month_6_cols = []
month_7_cols = []
for col in avg_6_7_columns:
    month_6_cols.append(str(col[:-4] + '_6'))
    month_7_cols.append(str(col[:-4] + '_7'))

In [ ]:
for i in range(0, len(avg_6_7_columns)):
    HVC_data[avg_6_7_columns[i]] = (HVC_data[month_6_cols[i]] + HVC_data[month_7_cols[i]])/2    

In [ ]:
HVC_data.head()

In [ ]:
HVC_data.shape

We drop the mobile_number column as it acts as unique ID and doest have any predictive value

In [ ]:
HVC_data.shape

In [ ]:
sorted(list(HVC_data.columns))

In [ ]:
# Lets check the correlation between independent variables with the target variable- 'Churn'
plt.figure(figsize=(6,36))
sns.heatmap(HVC_data.corr()[['Churn']].sort_values(by='Churn', ascending=False), annot=True)
plt.show()

In [ ]:
# lets check the correlation amongst the independent variables, drop the highly correlated ones
HVC_corr = HVC_data.corr()

#Stacking the columns into single column for easy readability
HVC_corr = HVC_corr.unstack()

#Filtering the highly correlated columns
HVC_corr = HVC_corr[((HVC_corr > 0.85) & (HVC_corr < 1)) | ((HVC_corr < -0.85) & (HVC_corr > -1))].sort_values(ascending=False) 

HVC_corr

In [ ]:
# Removing some high related columns above 86% based on the corelation and business knowledge
high_correl_cols = ['sachet_2g_8','sachet_2g_6_7','isd_og_mou_8',
                   'loc_ic_mou_8','loc_ic_mou_6_7','sachet_3g_8','sachet_3g_6_7',] 
HVC_data.drop(high_correl_cols, axis=1, inplace=True)
HVC_data.shape

In [ ]:
HVC_data.head()

In [ ]:
for i in HVC_data.columns:
    print(i)

In [ ]:
HVC_data['onnet_min']=HVC_data['onnet_mou_6']+HVC_data['onnet_mou_7']+HVC_data['onnet_mou_8']
HVC_data['offnet_min']=HVC_data['offnet_mou_6']+HVC_data['offnet_mou_7']+HVC_data['offnet_mou_8']
HVC_data.drop(['onnet_mou_6','onnet_mou_7','onnet_mou_8','offnet_mou_6','offnet_mou_7','offnet_mou_8'],1,inplace=True)

In [ ]:
mou=[]
for i in HVC_data.columns:
    if 'mou' in i:
        mou.append(i)
mou

In [ ]:
HVC_data.drop(mou,1,inplace=True)

In [ ]:
HVC_data.shape

In [ ]:
HVC_data.info()

In [ ]:
HVC_data.shape

In [ ]:
plt.figure(figsize = (50, 50))
sns.heatmap(HVC_data.corr())
plt.show()

In [ ]:
HVC_data.info()

In [ ]:
churn_rate = (sum(HVC_data["Churn"])/len(HVC_data["Churn"].index))*100
churn_rate

In [ ]:
#X_train.iloc[:,31:34]=X_train.iloc[:,31:34].astype('int64')
#X_train.iloc[:,25:26]=X_train.iloc[:,25:26].astype('int64')
#X_train.iloc[:,13:14]=X_train.iloc[:,13:14].astype('int64')
#X_train.iloc[:,19:20]=X_train.iloc[:,19:20].astype('int64')
#X_train.iloc[:,16:17]=X_train.iloc[:,16:17].astype('int64')

## Step-5 :
### Data modeling and evaliation 

### Split data in to train and test sets

In [ ]:
## Creating X and y variables
X = HVC_data.drop(['Churn'],axis=1)
y = HVC_data['Churn']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# splitting the dateset into train and test datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train.describe().T

In [ ]:
X_train.info()

## Performing scaling

In [ ]:
# Scaling the features
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train[X_train.columns] = scaler.fit_transform(X_train[X_train.columns])

In [ ]:
# check version number
import imblearn
print(imblearn.__version__)

In [ ]:
X_train.info()

## handling data imbalence

In [ ]:
# Data Imbalance Handling
# we employ SMOTE method to handle the data imbalance with target variable
import imblearn
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_smote,y_train_smote = smote.fit_resample(X_train,y_train)

In [ ]:
print(X_train_smote.shape)
print(y_train_smote.shape)

We'll use these data sets X_train_smote and y_train_smote throughout various models

## Logistic Regression

### Model-1

In [ ]:
# Importing necessary libraries for Model creation
import statsmodels.api as sm

log_reg_1 = sm.GLM(y_train_smote,(sm.add_constant(X_train_smote)), family = sm.families.Binomial())
log_reg_1.fit().summary()

## Logistic Regression with RFE Features
## Model-2

In [ ]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
log_reg_rfe = LogisticRegression()


from sklearn.feature_selection import RFE

# running RFE with 25 variables as output
rfe = RFE(log_reg_rfe, n_features_to_select=25)             
rfe = rfe.fit(X_train_smote, y_train_smote)

In [ ]:
rfe_columns = X_train_smote.columns[rfe.support_]
rfe_columns

In [ ]:
list(zip(X_train_smote.columns, rfe.support_, rfe.ranking_))

Apply manual method to evaluate the features

In [ ]:
# logistic regression model_2
X_train_smote_2 = sm.add_constant(X_train_smote[rfe_columns])
log_reg_2 = sm.GLM(y_train_smote,X_train_smote_2, family = sm.families.Binomial())
res = log_reg_2.fit()
res.summary()

As looking at the  p-values all are in the acceptable range (i.e. p-value > 0.05). Hence,no droping of any variable. Before procede to the next step we can check the VIFs

## VIF_2
checking the VIF values for model_2

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_smote_2[rfe_columns].columns
vif['VIF'] = [variance_inflation_factor(X_train_smote_2[rfe_columns].values, i) for i in range(X_train_smote_2[rfe_columns].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

based on above vif table the high vif value variable arpu_6_7 with 	93.21 value .But we can go on with  its p-value is > 0.05.

## Model - 3

In [ ]:
# logistic regression model_3
X_train_smote_3 = sm.add_constant(X_train_smote[rfe_columns])
log_reg_3 = sm.GLM(y_train_smote,X_train_smote_3, family = sm.families.Binomial())
res = log_reg_3.fit()
res.summary()


p-values are remain within the acceptable limits (p-value < 0.05)

## VIF_3
checking the VIF values for model_3

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_smote_3[rfe_columns].columns
vif['VIF'] = [variance_inflation_factor(X_train_smote_3[rfe_columns].values, i) for i in range(X_train_smote_3[rfe_columns].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

From the above table of VIF, the variable 'overall_rech_amt_6_7' holds different VIF value, hence, we drop this variable to reduce the multi-collinearity

In [ ]:
rfe_columns_3=rfe_columns.drop("overall_rech_amt_6_7",1)

## Model - 4

In [ ]:
# logistic regression model_4
X_train_smote_4 = sm.add_constant(X_train_smote[rfe_columns_3])
log_reg_4 = sm.GLM(y_train_smote,X_train_smote_4, family = sm.families.Binomial())
res = log_reg_4.fit()
res.summary()

## VIF_4
checking the VIF values for model_4

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train_smote_4[rfe_columns_3].columns
vif['VIF'] = [variance_inflation_factor(X_train_smote_4[rfe_columns_3].values, i) for i in range(X_train_smote_4[rfe_columns_3].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

All VIF values are within the acceptable limits(~5 or less)except some importent features for the future modeling. Hence, the resultant features are significant without any multi-collinearity

In [ ]:
X_train_smote_4.shape

In [ ]:
# Getting the predicted values on the train set
y_train_pred = res.predict(X_train_smote_4)
y_train_pred =y_train_pred.values.reshape(-1)
y_train_pred[:10]

 Creating a dataframe with the actual churn flag and the predicted probabilities

In [ ]:
y_train_pred_final = pd.DataFrame({'Churn':y_train_smote.values, 'Churn_Prob':y_train_pred})
y_train_pred_final['CustID'] = y_train_smote.index
y_train_pred_final.head()

In [ ]:
 #Creating new column 'predicted' with 1 if Churn_Prob > 0.5 else 0
y_train_pred_final['predicted'] = y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.5 else 0)

# Let's see the head
y_train_pred_final.head()

In [ ]:
# Let's take a look at the confusion matrix again 
from sklearn import metrics
confusion = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.predicted )
confusion

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))

In [ ]:
## Metrics beyond simply accuracy
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Calculate sensitivity 
print("Sensitivity = ",TP / float(TP+FN))

# Calculate specificity
print("Specificity = ",TN / float(TN+FP))

# Calculate False Positive Rate
print("False Positive Rate = ",FP/ float(TN+FP))

# Calculate Precision
print ("Precision = ",TP / float(TP+FP))

# Caclulate Negative predictive value
print ("Negative Prediction Rate = ",TN / float(TN+FN))

## Plotting the ROC Curve

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.Churn, y_train_pred_final.Churn_Prob, drop_intermediate = False )

In [ ]:
draw_roc(y_train_pred_final.Churn, y_train_pred_final.Churn_Prob)

## Finding Optimal Cutoff Point

In [ ]:
# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
from sklearn.metrics import confusion_matrix

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.show()

From the curve above, 0.5 is the optimum point to take it as a cutoff probability hence we will not change the optimum point

## Making predictions on the test set

In [ ]:
# Scaling the test data
X_test[X_train.columns] = scaler.transform(X_test[X_train.columns])
X_test_df = X_test.copy() # For future use

In [ ]:
# Feature selection
X_test=X_test[rfe_columns_3]

# Adding constant to the test model.
X_test_SM = sm.add_constant(X_test)


In [ ]:
# Predicting the target variable
y_test_pred = res.predict(X_test_SM)
y_pred = pd.DataFrame(y_test_pred)
y_pred=y_pred.rename(columns = {0:"Prob"})
y_pred.head()

In [ ]:
y_test_df = pd.DataFrame(y_test)
y_pred_final = pd.concat([y_test_df,y_pred],axis=1)

In [ ]:
y_pred_final['test_churn_pred'] = y_pred_final.Prob.map(lambda x: 1 if x>0.54 else 0)
y_pred_final.head()

In [ ]:
# Checking the overall accuracy of the predicted set.
metrics.accuracy_score(y_pred_final.Churn, y_pred_final.test_churn_pred)

In [ ]:
# Confusion Matrix
confusion2_test = metrics.confusion_matrix(y_pred_final.Churn, y_pred_final.test_churn_pred)
confusion2_test

In [ ]:
## Metrics beyond simply accuracy
TP2 = confusion2_test[1,1] # true positive 
TN2 = confusion2_test[0,0] # true negatives
FP2 = confusion2_test[0,1] # false positives
FN2 = confusion2_test[1,0] # false negatives

In [ ]:
# Calculate sensitivity 
print("Sensitivity = ",TP2 / float(TP2+FN2))

# Calculate specificity
print("Specificity = ",TN2 / float(TN2+FP2))

# Calculate False Positive Rate
print("False Positive Rate = ",FP2/ float(TN2+FP2))

# Calculate Precision
print ("Precision = ",TP2 / float(TP2+FP2))

# Caclulate Negative predictive value
print ("Negative Prediction Rate = ",TN2 / float(TN2+FN2))

##  With PCA
Model-1
We already have the balanced train data in X_train_smote and y_train_smote

In [ ]:
print(X_train_smote.shape)
print(y_train_smote.shape)
print(X_test_df.shape)
print(y_test_df.shape)


In [ ]:
# importing PCA
from sklearn.decomposition import PCA
pca = PCA(random_state=48)

# applying PCA on train data
pca.fit(X_train_smote)

In [ ]:
X_train_smote_pca = pca.fit_transform(X_train_smote)
X_test_pca = pca.transform(X_test_df)

In [ ]:
# Building Logistic Regression using all principal components

log_reg_pca_all = LogisticRegression()
log_reg_pca_all.fit(X_train_smote_pca, y_train_smote)

# making the predictions
y_pred = log_reg_pca_all.predict(X_test_pca)

# converting the prediction into a dataframe
y_pred_df = pd.DataFrame(y_pred)

Evaluating the performance measures

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
# Checking the Confusion matrix
conf_matrix_pca_all = confusion_matrix(y_test_df,y_pred)

# Checking the Accuracy of the Predicted model.
accuracy_pca_all = accuracy_score(y_test_df,y_pred)

In [ ]:
print(conf_matrix_pca_all)
print(accuracy_pca_all)

In [ ]:
# checking the components of pca
pca.components_

In [ ]:
cumul_variance = np.cumsum(pca.explained_variance_ratio_)

# Making a scree plot
fig = plt.figure(figsize=[16,10])
plt.plot(cumul_variance)
plt.xlabel('No of Principal Components')
plt.ylabel('Cumulative Variance')
plt.show()

In [ ]:
np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

From the above cumulative variance numbers, it is observed that the 17 components explains about 95% of variance in dependent variable and 10 principal components explains about 90% of variance.
*We build the logistic regression model using these first 17 and 10 principal components seperately and evaluate the performance.

## Model-2

In [ ]:
pca_17 = PCA(n_components=17)
train_pca_17 = pca_17.fit_transform(X_train_smote)
test_pca_17 = pca_17.transform(X_test_df)

In [ ]:
log_reg_pca_17 = LogisticRegression()
log_reg_pca_17.fit(train_pca_17, y_train_smote)

# making the predictions
y_pred_17 = log_reg_pca_17.predict(test_pca_17)

# converting the prediction into a dataframe
y_pred_17_df = pd.DataFrame(y_pred_17)

In [ ]:
# Checking the Confusion matrix
conf_matrix_pca_17 = confusion_matrix(y_test,y_pred_17)
accuracy_pca_17 = accuracy_score(y_test,y_pred_17)

In [ ]:
print(conf_matrix_pca_17)
print(accuracy_pca_17)

The accuracy obtained by 17 principal components is significant and reduced only marginally compared to that of model from all the components.

Model-3

In [ ]:
pca_10 = PCA(n_components=10)

train_pca_10 = pca_10.fit_transform(X_train_smote)
test_pca_10 = pca_10.transform(X_test_df)

In [ ]:
log_reg_pca_10 = LogisticRegression()
log_reg_pca_10.fit(train_pca_10, y_train_smote)

# making the predictions
y_pred_10 = log_reg_pca_10.predict(test_pca_10)

# converting the prediction into a dataframe
y_pred_10_df = pd.DataFrame(y_pred_10)

In [ ]:
# Checking the Confusion matrix
conf_matrix_pca_10 = confusion_matrix(y_test,y_pred_10)
accuracy_pca_10 = accuracy_score(y_test,y_pred_10)

In [ ]:
print(conf_matrix_pca_10)
print(accuracy_pca_10)

The accuracy measure though acceptable has dropped significantly from the base model. Hence, we consider the model with 17 principal components.

## Random Forest
We already have the balanced train in X_train_smote and y_train_smote

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train_smote,y_train_smote)

In [ ]:
y_pred_rfc = rfc.predict(X_test_df)

In [ ]:
# Checking the Confusion matrix
conf_matrix_rfc = confusion_matrix(y_test,y_pred_rfc)
accuracy_rfc = accuracy_score(y_test,y_pred_rfc)

In [ ]:
# Checking the Confusion matrix
conf_matrix_rfc = confusion_matrix(y_test,y_pred_rfc)
accuracy_rfc = accuracy_score(y_test,y_pred_rfc)

In [ ]:
print(conf_matrix_rfc)
print(accuracy_rfc)

In [ ]:
y_pred_rfc_best = rfc.predict(X_test_df)

In [ ]:
# Checking the Confusion matrix
conf_matrix_rfc_best = confusion_matrix(y_test,y_pred_rfc_best)
accuracy_rfc_best = accuracy_score(y_test,y_pred_rfc_best)

print(conf_matrix_rfc_best)
print(accuracy_rfc_best)

Note that the best parameters procuded the accuracy of 91% which is not significantly deterred than the accuracy of original random forest, which is pegged around 92%

Conclusion
The best model to predict the churn is observed to be Random Forest based on the accuracy as performance measure.

The incoming calls (with local same operator mobile/other operator mobile/fixed lines, STD or Special) plays a vital role in understanding the possibility of churn. Hence, the operator should focus on incoming calls data and has to provide some kind of special offers to the customers whose incoming calls turning lower.

Notes:
After cleaning the data, we broadly employed three models as mentioned below including some variations within these models in order to arrive at the best model in each of the cases.

Logistic Regression
Logistic Regression with RFE
Logistic regression with PCA
Random Forest
For each of these models, the summary of performance measures are as follows:

Logistic Regression

Accuracy: 78%

Logistic regression with PCA

Accuracy (all principal components includes): ~78%
Accuracy (all principal components includes): ~74%
Accuracy (all principal components includes): ~60%

Random Forest

Accuracy (without hyperparameter tuning): ~92%
Accuracy (with hyperparameter tuning): ~91%